# Key/Value RDD

---

이번 notebookt에서는 Spark의 Key/Vlaue RDD 이어서 학습합니다. 

Key/Vlaue RDD가 기억이 안 난다면, **HW3의 내용을 다시 복습 후에 진행하시길 권장**합니다. 


* task 별로 부분점수가 있습니다.(task 가 없는 경우는 부분점수가 없습니다(Exercise 1,2))

* 과제 제출기간 : 10월 1일 실습시간부터 ~ 10월 7월 오후 23:59분까지(+7일)

 - - -
 
**과제 점수 부여방식**

 * 데드라인(실습시간으로부터 +7일) 이후 제출시, +8 ~ +10일 까지는 해당 과제 점수의 20% 차감, +11일부터는 0점
 * 데드라인 이후 제출시, sempre813@naver.com로 학번_이름을 포함하여 보내주세요. (메일 보낸시간 == 제출 시간)
 * Exercise 수행시 주석이 없을 경우, 해당 Exercise 0점
 * copy and paste 발견시 예외없이 해당 주차 과제 0점(보여준 사람 포함)

### pyspark import & SparkContext 생성(이제는 무엇인지 알아야 한다..)

In [4]:
from pyspark import SparkContext
sc = SparkContext(master="local[*]")
sc
#sc.stop()

<SparkContext master=local[*] appName=pyspark-shell>

## Key-Value Transformations



(1) **keys() : key/value RDD에서 키가 있는 RDD 또는 key/value RDD 각 튜플의 첫 번째 요소(key)를 반환.**

In [8]:
kvpairs = sc.parallelize([('city', 'Hayward'), ('state', 'CA'), ('zip', 94541), ('country', 'USA')])

kvpairs.keys().collect()

['city', 'state', 'zip', 'country']

(2) **values() : key/value RDD에서 키가 있는 RDD 또는 key/value RDD 각 튜플의 두 번째 요소(value)를 반환**

In [9]:
kvpairs = sc.parallelize([('city', 'Hayward'), ('state', 'CA'), ('zip', 94541), ('country', 'USA')])

kvpairs.values().collect()

['Hayward', 'CA', 94541, 'USA']

(3) **keyBy(function) : function 인수로 지정된 함수를 적용해 RDD에 있는 요소의 key와 value로 구성된 튜플을 만든다.**

In [10]:
locations = sc.parallelize([('Hayward', 'USA', 1), ('Baumholder', 'Germany', 2), ('Alexandria', 'USA', 3), 
                           ('Melbourne', 'Australia', 4)])

bylocno = locations.keyBy(lambda x: x[2])
bylocno.collect()

[(1, ('Hayward', 'USA', 1)),
 (2, ('Baumholder', 'Germany', 2)),
 (3, ('Alexandria', 'USA', 3)),
 (4, ('Melbourne', 'Australia', 4))]

## Exercise 1 (10 point)

- - -
 
다음 RDD에 대하여 다음 과제를 수행하세요.

```
sc.parallelize([('Hayward', 'USA', 1), ('Baumholder', 'Germany', 2), ('Alexandria', 'USA', 3),('Melbourne', 'Australia', 4)])
```

`keyBySelect`라는 함수를 작성

- input: RDD of tuple
- output : **국가명이 Key**, **도시명이 Value**인 key/value RDD(**tip.map 또는 mapValue를 사용**)

`keyBySelect(tupleRDD)`가 다음을 출력하도록 합니다.

```
[('USA', 'Hayward'), ('Germany', 'Baumholder'), ('USA', 'Alexandria'), ('Australia', 'Melbourne')]
```



In [25]:
# 여기에 코드를 입력하세요
def keyBySelect(tupleRDD):
    
    print(locations.mapValues(lambda x : x).collect()) 
    # locations RDD에 mapValues을하여 key의 값과는 관계 없이 value에만 function을 적용

keyBySelect(locations)

[('Hayward', 'USA'), ('Baumholder', 'Germany'), ('Alexandria', 'USA'), ('Melbourne', 'Australia')]


(3) **subtractByKey(RDD) : RDD1, RDD2가 주어졌을 때, RDD1에서 RDD2와 같은 key 가질 경우 해당 요소 삭제**

In [0]:
rdd1 = sc.parallelize([(1,2),(2,1),(2,2)])
rdd2 = sc.parallelize([(2,5),(3,1)])

print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print("subtractByKey_Result:", rdd1.subtractByKey(rdd2).collect())

rdd1= [(1, 2), (2, 1), (2, 2)]
rdd2= [(2, 5), (3, 1)]
subtractByKey_Result: [(1, 2)]


### ``join``에 관련된 간단한 설명 (해석하는 것이 힘들다면.. [여기클릭](https://futurists.tistory.com/17))

There are four variants of `join` which differ in how they treat keys that appear in one dataset but not the other.
* `join` is an *inner* join which means that keys that appear only in one dataset are eliminated.
* `leftOuterJoin` keeps all keys from the left dataset even if they don't appear in the right dataset. The result of leftOuterJoin in our example will contain the keys `John, Jill, Kate`
* `rightOuterJoin` keeps all keys from the right dataset even if they don't appear in the left dataset. The result of leftOuterJoin in our example will contain the keys `Jill, Grace, John`
* `FullOuterJoin` keeps all keys from both datasets. The result of leftOuterJoin in our example will contain the keys `Jill, Grace, John, Kate`

In outer joins, if the element appears only in one dataset, the element in `(K,(V,W))` that does not appear in the dataset is represented bye `None`


(4) **(inner) join**

* A fundamental operation in relational databases.
* assumes two tables have a **key** column in common. 
* merges rows with the same key.

|**dataset 1**|                                     |..........| **dataset 2** | 	       	     |
|-------------|-------------------------------------|   |-------------|-----------------|
| **key=name**   |   **(gender,occupation,age)**    |   |  **key=name**   |   **hair color**    |
| John   |  (male,cook,21)                          |   | Jill   |  blond |
| Jill   |  (female,programmer,19)                  |   | Grace  |  brown |         
| John   |  (male, kid, 2)                          |   | John   |  black |
| Kate   |  (female, wrestler, 54)                  |

When `Join` is called on datasets of type `(Key, V)` and `(Key, W)`, it  returns a dataset of `(Key, (V, W))` pairs with all pairs of elements for each key. Joining the 2 datasets above yields:


|   key = name | (gender,occupation,age),haircolor |
|--------------|-----------------------------------|
| John         | ((male,cook,21),black)             |
| John         | ((male, kid, 2),black)             |
| Jill         | ((female,programmer,19),blond)     |


In [0]:
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print("Join_Result:", rdd1.join(rdd2).collect())

rdd1= [(1, 2), (2, 1), (2, 2)]
rdd2= [(2, 5), (3, 1)]
Join_Result: [(2, (1, 5)), (2, (2, 5))]


(5) **leftOuterJoin**

In [0]:
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print("Result:", rdd1.leftOuterJoin(rdd2).collect())

rdd1= [(1, 2), (2, 1), (2, 2)]
rdd2= [(2, 5), (3, 1)]
Result: [(1, (2, None)), (2, (1, 5)), (2, (2, 5))]


(6) **rightOuterJoin**

In [0]:
print('rdd1=',rdd1.collect())
print('rdd2=',rdd2.collect())
print("Result:", rdd1.rightOuterJoin(rdd2).collect())

rdd1= [(1, 2), (2, 1), (2, 2)]
rdd2= [(2, 5), (3, 1)]
Result: [(2, (1, 5)), (2, (2, 5)), (3, (None, 1))]


## Exercise 2 (10 point)

- - -
 
다음 RDDs에 대하여 다음 과제를 수행하세요.

```
stores = sc.parallelize([(100, 'Boca Raton'), (101, 'Columbia'), (102, 'Cambridge'), (103, 'Naperville')])
salespeople = sc.parallelize([(1, 'Henry', 100), (2, 'Karen', 100), (3, 'Paul', 101), (4, 'Jimmy', 102), (5, 'Janice', None)])
```

- input: stores, salepeople(RDD)
- output : `joinResult` 결과

`joinResult(RDD1, RDD2)`가 다음을 출력하도록 합니다.

```
[(100, ((1, 'Henry', 100), 'Boca Raton')), (100, ((2, 'Karen', 100), 'Boca Raton')), (101, ((3, 'Paul', 101), 'Columbia')), (102, ((4, 'Jimmy', 102), 'Cambridge'))]
```



In [57]:
# 여기에 코드를 입력하세요.
stores = sc.parallelize([(100, 'Boca Raton'), (101, 'Columbia'),(102, 'Cambridge'),(103, 'Naperville')])
salespeople = sc.parallelize([(1,'Henry',100),(2,'Karen',100),(3,'Paul',101),(4,'Jimmy',102),(5,'Janice',None)])

def joinResult(RDD1, RDD2):
    RDD = salespeople.keyBy(lambda x : x[2]) 
    # keyBy함수를 써서 salespeople의 2번째 인덱스를 key로 사용한다고 지정 후 RDD라는 변수에 저장
    print(RDD1.join(RDD).collect()) # RDD1인 stores와 RDD를 조인
joinResult(stores,salespeople)


[(100, ('Boca Raton', (1, 'Henry', 100))), (100, ('Boca Raton', (2, 'Karen', 100))), (101, ('Columbia', (3, 'Paul', 101))), (102, ('Cambridge', (4, 'Jimmy', 102)))]


## Key-Value Action

In [0]:
rdd = sc.parallelize([(1,2), (2,4), (2,6)])

#### 1. countByKey(): 
Count the number of elements for each key. Returns a dictionary for easy access to keys.

In [0]:
print("RDD: ", rdd.collect())
result = rdd.countByKey()
print("Result:", result)

RDD:  [(1, 2), (2, 4), (2, 6)]
Result: defaultdict(<class 'int'>, {1: 1, 2: 2})


#### 2. collectAsMap(): 
Collect the result as a dictionary to provide easy lookup.

In [0]:
print("RDD: ", rdd.collect())
result = rdd.collectAsMap()
print("Result:", result)

RDD:  [(1, 2), (2, 4), (2, 6)]
Result: {1: 2, 2: 6}


#### 3. lookup(key): 
Return all values associated with the provided key.

In [0]:
print("RDD: ", rdd.collect())
result = rdd.lookup(2)
print("Result:", result)

RDD:  [(1, 2), (2, 4), (2, 6)]
Result: [4, 6]


## Exercise 3 - Key/Value RDD 종합1 (20 point)

- - -
 
다음 데이터에 대하여 다음 과제를 수행하세요.

- regular.csv : KBO에서 활약한 타자들의 역대 정규시즌 성적을 포함하여 몸무게, 키 ,생년월일 등의 기본정보

**위의 두 데이터는 모두 `,`로 구분되어 있습니다.**

 - **데이터의 자세한 설명은 다음의 링크를 참조해주세요.([여기를 눌러서 12. 데이터 설명 참고](https://dacon.io/cpt6/62885))**
 - 또한 regular.csv와 pre.csv를 직접 열어서 데이터가 어떻게 저장되어 있는지 확인해주세요.

- - -

**`kboplayerName(task)` :역대 정규시즌 성적에서 타율(avg)이 3할(0.300)을 8번 이상 기록했던 선수의 이름을 출력하는 함수.**

- input: sc.testFile를 이용하여 regular.csv를 RDD로 생성
- output : `kboplayerName` 결과

**task**

`kboplayerName(task)` 함수는 다음의 기능을 담고 있습니다.

- 1. map을 사용하여 key(타자 이름)/value(타율)의 RDD로 transformation 합니다. (5 point)

 **단, value(tuple)의 타입은 float이 되도록 합니다. 또한, 타율 항목이 ''(공백)인 경우 0으로 처리합니다.** 
 
 
- 2. map을 사용하여, if value == '' return (key, 0) else (key, float(value)인 RDD로 transformation 합니다. (5 point)
- 3. filter를 사용하여 value가 0.300 이상인 RDD로 transformation 합니다. (5 point)
- 4. countByKey를 사용하여 3할을 8번 이상 기록했던 선수의 이름을 출력합니다. (5 point) 

 [python dictionary 참고](https://junho85.pe.kr/671)



---



`kboplayerName(task)`가 다음을 출력하도록 합니다.

```
['김동주', '김주찬', '김태균', '김현수', '박용택', '박한이', '손아섭', '이대호', '이진영', '이택근', '장성호', '정근우', '정성훈', '최형우']
```


In [5]:
import urllib.request
import re

f = urllib.request.urlretrieve ("https://docs.google.com/uc?export=download&id=1XaYz8IPIlxsWia59RUdTJtfkLNc2MJqk", "pre.csv")
f = urllib.request.urlretrieve ("https://docs.google.com/uc?export=download&id=1b_L-rJYJC9Oqga0fQ2zh2M763CTM8jzR", "regular.csv")

In [166]:
regular = "./regular.csv"
regular = sc.textFile(regular)

# 여기에 코드를 입력하세요
def kboplayerName(task):
    
    RDD = task.map(lambda x : (x.split(",")[1], x.split(",")[4])) 
    # split을 사용하여 ","로 나누어진 데이터의 1번째를 key로 4번째를 value로 map을 하여 transformation 한후 RDD 변수에 저장
    RDD2 = RDD.map(lambda x : (x[0], 0 if x[1] == "" else (float(x[1])))).filter(lambda x : x[1] >= 0.300).countByKey()
    # 만약 RDD의 1번째 인자가 공백일 경우 0으로 처리 그렇지 않으면 실수형으로 형변환 한 후 1번째 인자가 0.300이상인 데이터만 filter를 
    # 사용하여 추출한 후 countByKey를 사용하여 추출된 키 값을 센후 RDD2에 저장
    
    for key, value in RDD2.items(): #RDD2의 데이터 만큼 key value를 반복하는 반복문
        if value >= 8: # value의 값이 8이상인 경우            
            print(list(key)) # 그 조건에 만족하는 key값을 리스트 형태로 출력
      
    #print(RDD2)
kboplayerName(regular)  

['김', '동', '주']
['김', '주', '찬']
['김', '태', '균']
['김', '현', '수']
['박', '용', '택']
['박', '한', '이']
['손', '아', '섭']
['이', '대', '호']
['이', '진', '영']
['이', '택', '근']
['장', '성', '호']
['정', '근', '우']
['정', '성', '훈']
['최', '형', '우']


## Exercise 4 - Key/Value RDD 종합 2 (20 point)

- - -
 
다음 데이터에 대하여 다음 과제를 수행하세요.

- regular.csv : KBO에서 활약한 타자들의 역대 정규시즌 성적을 포함하여 몸무게, 키 ,생년월일 등의 기본정보
- pre.csv : KBO에서 활약한 타자들의 **역대 시범경기(정규시즌 직전에 여는 연습경기)** 성적

**위의 두 데이터는 모두 `,`로 구분되어 있습니다.**

 - **데이터의 자세한 설명은 다음의 링크를 참조해주세요.([여기를 눌러서 12. 데이터 설명 참고](https://dacon.io/cpt6/62885))**
 - 또한 regular.csv와 pre.csv를 직접 열어서 데이터가 어떻게 저장되어 있는지 확인해주세요.

- - -

**`hitCompare(task)`** :

**역대 정규시즌의 평균 타율과 역대 시범경기 평균 타율을 비교하여, 역대 정규시즌 평균 타율이 역대 시범경기 타율보다 높은 선수의 이름을 출력하는 함수**

- input: sc.testFile를 이용하여 regular.csv, pre.csv를 각각 RDD로 생성
- output : `hitCompare` 결과

**task**

`hitCompare(task)` 함수는 다음의 기능을 담고 있습니다.

- 1. ``map``을 사용하여 타자 이름 - key / tuple(타수, 안타(1,2,3루타,홈런)) - value의 RDD로 transformation 합니다**(regular, pre 각각 생성)** (5 point)

 **단, value(tuple)의 타입은 float이 되도록 합니다. 또한, 타수 또는 안타 항목이 ''(공백)인 경우 0으로 처리합니다.  [참고](https://niceman.tistory.com/165)** 
```
[('가르시아', (183.0, 62.0)), ('강경학', (1.0, 0.0)), ('강경학', (86.0, 19.0)), ('강경학', (311.0, 80.0)), ...]
```

- 2. ``reduceByKey와 map``을 이용하여 **역대 정규시즌 평균 타율과 역대 시범경기 평균 타율을 구하여 RDD로 각각 transformation 합니다.** (5 point)

 **단, 타율 = (안타/타수)  공식을 적용할 것**
 
 ```
 [('가르시아', 0.33879781420765026),
 ('강구성', 0.125),
 ('백승룡', 0.1642512077294686),
 ('진갑용', 0.27565814574589853),
 ('강백호', 0.2903225806451613),
 ('강병식', 0.23601532567049807),
 ('강봉규', 0.2621798091411351),
 ('강승호', 0.24680851063829787),
 ('강정호', 0.298371335504886),
 ('고도현', 0.2727272727272727)]
 ```
 

- 3. task 1과 task 2에서 정제된 RDD(역대 정규시즌, 역대 시범경기)를 선수 이름(key)로 `join` 후 `filter`를 적용하여 **역대 정규시즌 평균 타율이 역대 시범경기 평균 타율보다 높은 선수의 ``이름-key / tuple(역대 정규시즌 평균 타율, 역대 시범경기 평균 타율)-value의 key/value RDD``로 transformation 합니다** (5 point)

- 4. task 3에서 생성된 RDD로부터 역대 정규시즌 평균 타율에 ``sortByKey``를 적용하여 상위 10명의 이름을 출력합니다. (5 point)



---



`hitCompare(task)`가 다음을 출력하도록 합니다.

```
[(0.38461538461538464, '장승현'),
 (0.36363636363636365, '전병우'),
 (0.33827893175074186, '이정후'),
 (0.33410538506079906, '박건우'),
 (0.3333333333333333, '김태진'),
 (0.33191489361702126, '구자욱'),
 (0.32515082171832743, '손아섭'),
 (0.3247439180537772, '김태균'),
 (0.32383536861148804, '박민우'),
 (0.3226377517149812, '김현수')]
```


In [197]:
pre = "./pre.csv"
regular = "./regular.csv"

RDD_list = []
RDD_list.append(sc.textFile(regular))
RDD_list.append(sc.textFile(pre))

# 여기에 코드를 입력하세요
def hitCompare(task):
    regularRDD = task.map(lambda x : x.split(",")[1], x.split(",")([6],[8])).map(lambda x: (x[0], 0 if x[1][0] == "" else float(x[1][0]), 0 if x[1][1] == "" else float(x[1][1])))    
    preRDD = task.map(lambda x : x.split(",")[1], x.split(",")([6],[8])).map(lambda x: (x[0], 0 if x[1] == "" else float(x[1]), 0 if x[1][1] == "" else float(x[1][1])))
    # 데이터를 ","단위로 split하여 1번째 인덱스를 key로 6,8번째 인덱스를 튜플로 하여 value로 만든 후 타수와 안타 중 공백이
    # 있을 경우 0으로 출력하고 그렇지 않으면 타수와 안타를 실수형으로 형변환을 2개의 if문을 사용 인덱싱한다.
    
    regularRDD_rdd = regularRDD.map(lambda x : x[0],(x[1][0],x[1][1])).reduceByKey(lambda x,y : y/x)
    preRDD_rdd = preRDD.map(lambda x : x[0],(x[1][0],x[1][1])).reduceByKey(lambda x,y : y/x)
    # regular RDD를 map하여 다시 새로운 RDD를 만들고 1번째 인덱스에 있는(타수,안타)를 인덱싱한 다음 reduceByKey로 통해 key값을 기준으로 
    # value인 튜플에 타수/안타를 한다.
                            
    joinRDD = regularRDD_rdd.leftOuterJoin(preRDD_rdd).filter(regularRDD_rdd > preRDD_rdd).sortByKey().take(10)
    # key값을 기준으로 join이니까 leftouterjoin을 한다. 그런후 filter를 통해 위에서 구해놓은 정규시즌 평균이 프리시즌 평균보다 큰 값을
    # 추출 한 후 sortByKey() action을 통해 정렬 한 후 take(10)으로 상위 10가지를 출력한다.
    

hitCompare(RDD_list)

AttributeError: 'list' object has no attribute 'map'

## Exercise 5 - Key/Value RDD 종합 3 (20 point)

- - -

기존에 진행했던 HW3의 Exercise 3-4 ``Moby Dic``에서 n-grams 찾기와 매우 유사합니다. 

주어진 데이터(문장)에서 **NLTK를 이용하여 명사를 추출 및 단어의 길이에 따라 가장 많이 발생되는 단어를 찾는 문제**입니다

- word(2-length) : ``` 안녕, 남자, 여자, 생각, 친구 ``` 등
    
- word(3-length) : ```부모님, 어머니, 아버지, 아이돌, 분위기``` 등

- word(4-length) : ```고등학교, 스트레스, 남자친구, 여자친구``` 등

이번에 사용할 데이터는 Nate 판(https://pann.nate.com/) 에서 10대 이야기``를 **Web crwalling**을 이용하여 30,000개의 글을 수집한 데이터입니다. 


**데이터 구성**

- 제목  작성일시  조회(수)  본문

    예시 : ``"감기때문에 너무무기력함..어떡해ㅠㅠㅠㅠㅠ	2018.12.03 21:54	조회21	당장 다음주가시험인데 하루종일 공부안하고 누워서 폰하고먹고자고 반복임..머리가 깨질듯이아프니까 공부할 마음이 1도없여 증말"``
    
    
**Task**
- 1. !pip 를 이용하여 jpype1, konlpy 모듈 설치합니다. 
- 2. 다음의 링크를 참조[[konlpy 설명](https://cceeddcc.tistory.com/8), Okt, Hannanum, Kkma 등 1개 선택]하여, **주어진 문장을 명사로 추출하되 한글(모음) 외에 ``숫자, 특수기호, 조회(위의 데이터 구성 참조)`` 를 삭제하는 ``removeFunctuation``을 작성합니다. 즉, 제목과 본문만을 이용함** (10 point)

- 3. removeFunctuation을 통해 명사로 변환된 데이터를 **단어의 길이에 따라 RDD를 생성**합니다.(list 이용 추천). **단, 단어의 길이는 2, 3, 4로 한정.** (5 point)

- 4. 출력 예시를 참조하여 출력 함수인 **``printOutput를 작성합니다.``** (5 point)



---



**출력 예시**

```
============ 10 most frequent word(2-length)

RANK	COUNT	WORD
1	7217: 	"사람"
2	6180: 	"진짜"
3	5858: 	"팬톡"
4	5364: 	"생각"
5	4388: 	"친구"
6	3443: 	"지금"
7	3289: 	"그냥"
8	2983: 	"남자"
9	2715: 	"여자"
10	2572: 	"우리"

============ 10 most frequent word(3-length)

RANK	COUNT	WORD
1	993: 	"갑자기"
2	949: 	"한기총"
3	835: 	"부모님"
4	827: 	"이야기"
5	810: 	"피해자"
6	714: 	"신천지"
7	641: 	"제페토"
8	586: 	"얼마나"
9	576: 	"아이돌"
10	562: 	"분위기"

============ 10 most frequent word(4-length)

RANK	COUNT	WORD
1	860: 	"남자친구"
2	671: 	"고등학교"
3	456: 	"여자친구"
4	443: 	"스트레스"
5	281: 	"대한민국"
6	253: 	"다이어트"
7	251: 	"우리나라"
8	195: 	"트와이스"
9	194: 	"시어머니"
10	163: 	"미세먼지"
```




In [175]:
# pip를 통한 konlpy module 설치
!pip install jpype1
!pip install konlpy

     |████████████████████████████████| 2.8MB 106kB/s eta 0:00:01
     |████████████████████████████████| 19.4MB 241kB/s eta 0:00:01


In [198]:
# Nate webcrawling data 30,000 row(page)

import urllib.request
import re
from konlpy.tag import Okt # 한국어 NLP

f = urllib.request.urlretrieve ("https://docs.google.com/uc?export=download&id=1qUQ1GhvkZURUYa_SZI53KrLl1gw6j3VK", "Ex4_data.txt")
data_file = "./Ex4_data.txt"

def removeFunctuation(text):
    text = re.sub("[^가-힣]|[조회]", " ", text) # 한글만 남김(숫자 및 특수기호 제외)
    okt = Okt() 
    KoNLP_text = okt.nouns(text) # 한글 명사 추출
    return KoNLP_text

# *****조교님 제가 이 문제를 풀어보려 노력을 하였으나 도저히 제 힘으로는 결과를 도출하기 힘들어서 인용했습니다.ㅠㅠ******

def printOutPut(n,freq_wordRDD):
    top=freq_wordRDD.take(10) # freq_wordRDD의 상위 10개 출력
    print("============= %d most frequent word(%d-length)" % (10, n + 2)) # 단어의 길이 만큼 빈도수를 출력해준다는 문구를 출력하는 출력문
    print("\nRANK\tCOUNT\tWORD")
    for i in range(10): # i부터 10까지 반복하는 반복문
        print('"%d\t%d: \t"%s'%(i+1,top[i][0], top[i][1]))      
word_RDD.append(sc.parallelize(data_file).map(removeFunctuation).flatMap(lambda x : [(word, 1) for word in x if len(word) == 2]))
word_RDD.append(sc.parallelize(data_file).map(removeFunctuation).flatMap(lambda x : [(word, 1) for word in x if len(word) == 3]))
word_RDD.append(sc.parallelize(data_file).map(removeFunctuation).flatMap(lambda x : [(word, 1) for word in x if len(word) == 4]))
# flatmap() 은 하나의 입력에 출력이 여려개인 경우 사용하는데 이 경우 word의 단어 개수 추출이 하나가 들어가면 여려개를 추출하기 때문에
# map()아닌 flatmap을 사용했다.

for index in range(len(word_RDD)): # index부터 word_RDD의 길이 만큼 반복하는 반복문
    freq_word = RDD[index].reduceByKey(lambda x,y : x + y).map(lambda x : (x[1], x[0])).sortByKey(False)
    #RDD에 reduceByKey를하여 key값으로 value에 대한 덧셈 연산을 하고. map()을하여 새로운 RDD를 생성한 후 sortByKey에 인자로 False를 주어
    #역으로 정렬한다.
    printOutPut(index,freq_word)

# 여기에 코드를 입력하세요

NameError: name 'word_RDD' is not defined

## Exercise 6 - Key/Value RDD 종합 4 (20 point)
- - -

Exercise 4에서 사용했던 데이터를 다시 활용합니다.

- word(2-length) : ``` 안녕, 남자, 여자, 생각, 친구 ``` 등
    
- word(3-length) : ```부모님, 어머니, 아버지, 아이돌, 분위기``` 등
    
    
**Task**
- 1. count가 계산된 정제된 word(2-length), word(3-length)에서 **count가 100이상**인 word를 각각의 RDD(RDD1, RDD2)로 생성합니다 **(tip. filter)** (5 point)
- 2. input을 RDD list(RDD1과 RDD2가 포함된)로 받고, RDD_list를 join(inner) & 중복제거 후, word를 return 하는 **joinWordPrint**를 작성합니다.**(tip. flatMap, distinct)** (10 point)
- 3. 추출된 10개의 word를 이용하여 자신만의 문장을 만듭니다 (5 point)



---



**출력 예시**
- 1. joinWordPrint를 사용하여 추출된 word 출력

    ```['남녀', '아파트', '집착', '학기', '걔네', '상관', '그림', '시청', '미래', '펜션']```
    
    
- 2. 나만의 문장 생성(예시 문장) 반드시 **! 선택된 단어는 [ ] 마킹할 것**

    ``` 우리 집에는 [쿠션]이 [남아]돈다. 누나가 [매번] 남자친구를 만나고 돌아올 때마다 [쿠션]을 가져온다. 오늘도 어김없이 누나는 왼쪽 겨드랑이에 [쿠션]을 끼고 [등장]하였다. 이번에는 [쿠션] 브랜드 중 [브랜드]의 [쿠션]이라는데.. [길이]가 내 키만큼 길다. 갑자기 누나는 남자친구를 만나야 한다며 짐만 내팽개치고 밖으로 나갔다.. 밤 10시인데.. 뭐가 있나보다.. 나는 알 수 없다.. [매번] 밤 10시에 나간 누나는 다음날 오후 2시가 돼서야 들어온다.. 뭘 하는지 참.. 누나 방 침대는 최근에 구매해서 등과 허리를 아주 편하게 해줘서 sleep 하기 아주 좋은 [환경]이다. 그래서 나는 매번 누나가 남자친구를 야심한 시간에 만나러 갈 때마다, 그곳은 나의 구역이다. 사람은 무언가를 때릴 때 스트레스가 해소가 된다고 한다. 원래 나는 '한 놈만 팬다'의 좌우명을 가지고 있었지만, 오늘은 모든 [쿠션]을 [차별]없이 모두 손봐주었다. 스트레스가 풀린다.. 갑자기 누나가 집으로 왔다. 이번엔 혼자가 아니다. "[방해]하지 말고.. [가라].. 5천원 줄테니..". 나는 나의 방으로 돌아왔다. -끝- ```

In [199]:
def joinWordPrint(word_list):
    # 여기에 코드를 입력하세요
    
    # 이 부분은 upload에 포함할 것
    return return_word.takeSample(False, 10) # takeSample은 랜덤 샘플링을 지원하는 메서드이다. 두번 째 인자 값을 지정하여 몇개를
    # 추출할지 정하고, 결과가 RDD가 아닌 리스트나 배열로 나타난다.

print(joinWordPrint(word_list))

NameError: name 'word_list' is not defined

* 나만의 문장 작성
: 우리 학교에는 서로에게 [집착]이 심한 [남녀] 커플이 있다. 서로 [집착] 심한탓에 [걔네] 둘은 종종 싸우곤한다. 그 중 하나가 저번 [학기]때 일이다. 종강을 맞이 하여 [걔네]둘이 [펜션]에 놀러가기로 했었나보다. [걔네] 둘은 수업이 끝나고 [시청]에서 만나 서울역에서 기차를 타고 [펜션]으로 향했다. 처음에는 빌딩들과 [아파트]들이 즐비한 전형적인 도시의 풍경이였지만 기차를 타고 교외로 나가다보니 [그림]같은 풍경들이 펼쳐 졌다. 그 둘은 쉴세 없이 풍경을 감상하고, 떠들며 즐겁게 여행을 하고 있었다. 그때 남자친구의 핸드폰이 울렸고, 남자는 재빨리 핸드폰을 들고 기차 화장실로 향했다. 그때부터 사건이 발단했다. 통화를 마치고온 남자는 자리에 앉자마자 여자친구의 의심과 [집착]이 시작됐다. 남자는 너랑 [상관]없는 전화라고 신경쓰지말라고 했다. 여자는 나랑 [상관]없는 전환데 왜 나가서 받느냐고 받아쳤다. 그렇게 그 둘은 [펜션]에 도착할 때 까지 싸움을 이어 나갔다. 결국엔 남자가 핸드폰을 보여주면서 사건은 끝났지만 심한 [집착]과 의심 때문에 둘은 점점 지쳐만 갔다. 과연 앞으로의 [미래]에는 이 사람과 계속 사귈 수 있을지 까지 생각하게 됐다. 물론 연인간 [집착]을 아예 안하기는 힘들지만 서로를 믿는다면 [집착]을 좀더 줄일 수 있을것이다. -끝-